In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import glob

In [ ]:
df = pd.DataFrame()
initial = "/content/drive/MyDrive/Colab Notebooks/" 
#initial = ""
files = glob.glob(initial + "data/*")
course_data = glob.glob(initial + "data/uc.berkeley.student.course.data.*.txt")
majors = glob.glob(initial + "data/uc.berkeley.student.majors.*.txt")
pathways = glob.glob(initial + "data/uc.berkeley.stem.*.txt")
passes = glob.glob(initial + "data/jacobs.*")

In [ ]:
# every student who holds a maker pass
maker_passes = pd.read_table(passes[0])
maker_passes = maker_passes[maker_passes['ppsk'] != 'MISSING']
maker_passes = maker_passes['ppsk'].unique()

# all students who took design innovation course
all_jacobs = student_courses[student_courses["course.department"] == "Design Innovation"]['ppsk'].unique()

In [ ]:
# Be aware, this cell is very resource intensive
course_data.sort()
student_courses = pd.DataFrame()
for f in course_data:
    student_courses = student_courses.append(pd.read_table(f))

In [ ]:
# minor data cleaning
student_courses = student_courses[~student_courses['ppsk'].isnull()]
student_courses = student_courses[student_courses['ppsk'] != -1]
for i in student_courses.columns:
    if student_courses[i].isnull().all():
        student_courses.drop(i,axis=1,inplace=True)
student_courses['ppsk'] = student_courses['ppsk'].astype(int)